# First model 

This model will try to predict wether pe is present or not on a single image (grey level)

## Imports

In [1]:
import numpy as np
from balance import *
from model import *

In [2]:
import torch
torch.cuda.empty_cache()

print(f'PyTorch version: {torch.__version__}')
print("GPU found :)" if torch.cuda.is_available() else "No GPU :(")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchsummary import summary

PyTorch version: 1.7.1
GPU found :)


# Load datasets

In [3]:
channel0_dataset = torch.load('datasets/channel0.pt')
channel1_dataset = torch.load('datasets/channel1.pt')
channel2_dataset = torch.load('datasets/channel2.pt')

## Simple CNN from scratch

In [4]:
class ModelV1(nn.Module):
    
    def __init__(self, input_channels):
        """Simple Convnet with 2 convolution layer + pooling, with 2 fully connected at the end"""
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32*62*62 , 120)
        self.fc2 = nn.Linear(120, 1)
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32*62*62) # Flatten
        x = F.relu(self.fc1(x))
        
        # Binary classification
        out = self.sigmoid(self.fc2(x))
        
        return out
    
summary(ModelV1(1).cuda(), (1, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 254, 254]             160
         MaxPool2d-2         [-1, 16, 127, 127]               0
            Conv2d-3         [-1, 32, 125, 125]           4,640
         MaxPool2d-4           [-1, 32, 62, 62]               0
            Linear-5                  [-1, 120]      14,761,080
            Linear-6                    [-1, 1]             121
           Sigmoid-7                    [-1, 1]               0
Total params: 14,766,001
Trainable params: 14,766,001
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.25
Forward/backward pass size (MB): 14.60
Params size (MB): 56.33
Estimated Total Size (MB): 71.18
----------------------------------------------------------------


## Split data into train and test

In [5]:
batch_size = 128

train_loader0, test_loader0 = train_test_split(channel0_dataset, ratio=0.2, batch_size=batch_size)
train_loader1, test_loader1 = train_test_split(channel1_dataset, ratio=0.2, batch_size=batch_size)
train_loader2, test_loader2 = train_test_split(channel2_dataset, ratio=0.2, batch_size=batch_size)

## Model Training

In [6]:
model_channel0 = ModelV1(input_channels=1).to(device)
model_channel1 = ModelV1(input_channels=1).to(device)
model_channel2 = ModelV1(input_channels=1).to(device)

In [ ]:
model_channel0, losses_0 = train(model_channel0, num_epoch=10, criterion=nn.BCELoss(), 
                       data_loader=train_loader0, optimizer=optim.SGD(model_channel0.parameters(), lr=0.001), device=device, './models/model_v1_channel0.h5')

model_channel1, losses_1 = train(model_channel1, num_epoch=10, criterion=nn.BCELoss(),
                       data_loader=train_loader1, optimizer=optim.SGD(model_channel1.parameters(), lr=0.001), device=device, './models/model_v1_channel1.h5')

model_channel2, losses_2 = train(model_channel2, num_epoch=10, criterion=nn.BCELoss(), 
                       data_loader=train_loader2, optimizer=optim.SGD(model_channel2.parameters(), lr=0.001), device=device, './models/model_v1_channel2.h5')

Training begins
Epoch [1/10], step [20/209], loss: 0.6952
Epoch [1/10], step [40/209], loss: 0.6939
Epoch [1/10], step [60/209], loss: 0.6919
Epoch [1/10], step [80/209], loss: 0.6932
Epoch [1/10], step [100/209], loss: 0.6927
Epoch [1/10], step [120/209], loss: 0.6952
Epoch [1/10], step [140/209], loss: 0.6910
Epoch [1/10], step [160/209], loss: 0.6940
Epoch [1/10], step [180/209], loss: 0.6948
Epoch [1/10], step [200/209], loss: 0.6975
Epoch [2/10], step [20/209], loss: 0.6930
Epoch [2/10], step [40/209], loss: 0.6934
Epoch [2/10], step [60/209], loss: 0.6942
Epoch [2/10], step [80/209], loss: 0.6951
Epoch [2/10], step [100/209], loss: 0.6924
Epoch [2/10], step [120/209], loss: 0.6931
Epoch [2/10], step [140/209], loss: 0.6932
Epoch [2/10], step [160/209], loss: 0.6981
Epoch [2/10], step [180/209], loss: 0.6932
Epoch [2/10], step [200/209], loss: 0.6934
Epoch [3/10], step [20/209], loss: 0.6931
Epoch [3/10], step [40/209], loss: 0.6927
Epoch [3/10], step [60/209], loss: 0.6927
Epoch 

## Test the model

In [ ]:
# Load models for evaluation
model_channel0 = ModelV1(input_channels=1)
model_channel0.load_state_dict(torch.load('./models/model_v1_channel0.h5'))
model_channel0.to(device)

model_channel1 = ModelV1(input_channels=1)
model_channel1.load_state_dict(torch.load('./models/model_v1_channel1.h5'))
model_channel1.to(device)

model_channel2 = ModelV1(input_channels=1)
model_channel2.load_state_dict(torch.load('./models/model_v1_channel2.h5'))
model_channel2.to(device)

In [ ]:
evaluate(model_channel0, test_loader0, device)
evaluate(model_channel1, test_loader1, device)
evaluate(model_channel2, test_loader2, device)